In [1]:
import pandas as pd
import numpy as np
import requests

In [ ]:
#load chart data
df_pieChartData=pd.read_csv('/content/averagecounts_per_genekbp_percountry_19062020_EPI_ISL_402125.csv', delimiter=',')

#fix country names
df_pieChartData=df_pieChartData.replace({"Country":"NewZealand"},"New Zealand")
df_pieChartData=df_pieChartData.replace({"Country":"NetherlandsL"},"Netherlands")
df_pieChartData=df_pieChartData.replace({"Country":"Wuhan-Hu-1"},"Wuhan")
df_pieChartData=df_pieChartData.replace({"Country":"Nanchang"},"NanChang")
df_pieChartData=df_pieChartData.replace({"Country":"DRC"},"Congo")
df_pieChartData=df_pieChartData.replace({"Country":"Korea"},"South Korea")
df_pieChartData=df_pieChartData.replace({"Country":"Northern Ireland"},"Ireland")
df_pieChartData=df_pieChartData.replace({"country":"JAM"},"Jamaica")
df_pieChartData=df_pieChartData.replace({"country":"SouthAfrica"},"South Africa")
df_pieChartData=df_pieChartData.replace({"country":"Romania "},"Romania")

#melt data into format for arc
df_pieChartData = pd.melt(df_pieChartData,
        id_vars=['Country'],
        value_vars=df_pieChartData.columns[1:])
df_pieChartData.rename(columns={"variable": "gene", "value": "num_mutation"}, inplace=True)
df_pieChartData=df_pieChartData.sort_values(by=['Country','gene'])
df_pieChartData.head()

,Country,gene,num_mutation
586,Algeria,E,93.395029
722,Algeria,M,0.000000
1402,Algeria,N,33.800106
1538,Algeria,ORF10,0.000000
42,Algeria,ORF1A,16.128203


In [3]:
#load data and fix country names
df_headMapData=pd.read_csv('/content/counts_per_genekbp_percountry_19062020_EPI_ISL_402125.csv')
df_headMapData=df_headMapData.replace({"Country":"NetherlandsL"},"Netherlands")
df_headMapData=df_headMapData.replace({"Country":"NewZealand"},"New Zealand")
df_headMapData=df_headMapData.replace({"Country":"Wuhan-Hu-1"},"Wuhan")
df_headMapData=df_headMapData.replace({"Country":"DRC"},"Congo")
df_headMapData=df_headMapData.replace({"Country":"Nanchang"},"NanChang")
df_headMapData=df_headMapData.replace({"Country":"Korea"},"South Korea")
df_headMapData=df_headMapData.replace({"Country":"Northern Ireland"},"Ireland")
df_headMapData=df_headMapData.replace({"country":"JAM"},"Jamaica")
df_headMapData=df_headMapData.replace({"country":"SouthAfrica"},"South Africa")
df_headMapData=df_headMapData.replace({"country":"Romania "},"Romania")

#change GISAID Ids (to be done at earlier stage by rasha to have uniform ids)
df_headMapData["ID"]=df_headMapData.index.astype('str')

#metl into fomat
df_headMapData = pd.melt(df_headMapData,
        id_vars=['Country','ID','Date'],
        value_vars=df_headMapData.columns[3:])
df_headMapData.rename(columns={'ID':'strain_id',"variable": "gene", "value": "num_mutation"}, inplace=True)


countries=df_headMapData["Country"].unique()
mindate=pd.Series([])

df_headMapData.loc[df_headMapData.Date.str.len() <=4,"Date"]=np.nan
#fill na dates with min country date
for i in range(0,(len(countries))):
  mindate[i]=df_headMapData[df_headMapData["Country"]==countries[i]].Date.dropna().min()
  temp=df_headMapData.loc[df_headMapData["Country"]==countries[i],"Date"];
  df_headMapData.loc[df_headMapData["Country"]==countries[i],"Date"]= temp.fillna((mindate[i]))


df_CoutryDate=pd.DataFrame({"Country":countries,"minDate":mindate})

#add global option (df_headMapData3) that contains mindate for sample foreach country
df_headMapData2=df_headMapData.groupby(["Country",'gene'],sort=False).mean().reset_index()
df_headMapData2["strain_id"]=df_headMapData2["Country"]
df_headMapData2["Country"]="Global"

df_headMapData3= df_headMapData2.merge(df_CoutryDate,
                             how='inner',
                             
                       left_on=["strain_id"],right_on=['Country']
                      )

df_headMapData3.rename(columns={'Country_x':'Country',"minDate": "Date"}, inplace=True)

df_headMapData3=df_headMapData3[["Country","strain_id","Date","gene","num_mutation"]]

df_headMapData3.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,Country,strain_id,Date,gene,num_mutation
0,Global,USA,2020-01-19,ORF1A,0.194943
1,Global,USA,2020-01-19,ORF1B,0.223603
2,Global,USA,2020-01-19,S,0.256778
3,Global,USA,2020-01-19,ORF3A,0.900806
4,Global,USA,2020-01-19,E,0.085850


In [ ]:
#load and fix data country names
df_secondPieChartData=pd.read_csv('/content/average_consequence_percountry_19062020_EPI_ISL_402125_pergene.csv', delimiter=',')
df_secondPieChartData=df_secondPieChartData.replace({"country":"NewZealand"},"New Zealand")
df_secondPieChartData=df_secondPieChartData.replace({"country":"NetherlandsL"},"Netherlands")
df_secondPieChartData=df_secondPieChartData.replace({"country":"Wuhan-Hu-1"},"Wuhan")
df_secondPieChartData=df_secondPieChartData.replace({"country":"Nanchang"},"NanChang")
df_secondPieChartData=df_secondPieChartData.replace({"country":"DRC"},"Congo")
df_secondPieChartData=df_secondPieChartData.replace({"country":"Korea"},"South Korea")
df_secondPieChartData=df_secondPieChartData.replace({"country":"Northern Ireland"},"Ireland")
df_secondPieChartData=df_secondPieChartData.replace({"country":"JAM"},"Jamaica")
df_secondPieChartData=df_secondPieChartData.replace({"country":"SouthAfrica"},"South Africa")
df_secondPieChartData=df_secondPieChartData.replace({"country":"Romania "},"Romania")

In [ ]:
#load and fix data country names
df_nbsamples=pd.read_csv('/content/number_of_samples_19062020_EPI_ISL_402125.csv')
df_nbsamples=df_nbsamples.replace({"Var1":"NewZealand"},"New Zealand")
df_nbsamples=df_nbsamples.replace({"Var1":"NetherlandsL"},"Netherlands")
df_nbsamples=df_nbsamples.replace({"Var1":"Wuhan-Hu-1"},"Wuhan")
df_nbsamples=df_nbsamples.replace({"Var1":"Nanchang"},"NanChang")
df_nbsamples=df_nbsamples.replace({"Var1":"DRC"},"Congo")
df_nbsamples=df_nbsamples.replace({"Var1":"Korea"},"South Korea")
df_nbsamples=df_nbsamples.replace({"Var1":"SouthAfrica"},"South Africa")
df_nbsamples=df_nbsamples.replace({"Var1":"Northern Ireland"},"Ireland")
df_nbsamples=df_nbsamples.replace({"Var1":"JAM"},"Jamaica")
df_nbsamples=df_nbsamples.replace({"Var1":"Romania "},"Romania")

In [ ]:
#Check new countries

df_countries=pd.read_csv('/content/country_metadata.csv')


countries= df_secondPieChartData["country"].unique();
#countries= df_secondPieChartData["country"].unique();
df_countries["Country"].shape

idx1 = pd.Index(df_countries["Country"])
idx2 = pd.Index(pd.Series(countries))

display(idx2.difference(idx1).values)
display(idx1.difference(idx2).values)

array([], dtype=object)

array([], dtype=object)

In [ ]:
#concat and save heatmap data
df_finalConcat = pd.concat([df_headMapData,df_headMapData3],ignore_index=True)

df_finalConcat["Date_Sample"]=df_finalConcat["Date"]+"_"+df_finalConcat["strain_id"]
df_finalConcat["strain_id"]=df_finalConcat["Date_Sample"]

df_finalConcat=df_finalConcat[["Country","strain_id","Date","gene","num_mutation"]]
df_finalConcat.to_csv("genomics_counts_geneperkbp_heatMap.csv",index=False)


In [ ]:
#save piechart1 data
df_pieChartData2=df_headMapData.groupby(['gene']).mean().reset_index()
df_pieChartData2["Country"]="Global"

df_pieChartData2=df_pieChartData2[["Country","gene","num_mutation"]]

df_pieChartFinal = pd.concat([df_pieChartData,df_pieChartData2],ignore_index=True) 
df_pieChartFinal.to_csv("genomics_avgcounts_geneperkbp_pieChart.csv",index=False)


In [ ]:
#save piechart2 data
df_secondPieChartData.to_csv("genomics_avgconsequence_percountry_pergene_pieChart.csv",index=False)

In [ ]:
#merge country metadata with nbsamples and save data
df_countries2= df_countries.merge(df_nbsamples,
                             how='left',
                             
                       left_on=["Country"],right_on=['Var1']
                      )
df_countries2.rename(columns={"Freq": "nbSample"}, inplace=True)
df_countries2=df_countries2[["Country","Lat","Long","nbSample"]]

df_countries2=df_countries2.groupby(['Country','Lat','Long'], sort=False,as_index=False).sum()
df_countries2.head()

,Country,Lat,Long,nbSample
0,Global,0.000000,0.0000,48218
1,Algeria,28.033900,1.6596,3
2,Anhui,31.825700,117.2264,1
3,Argentina,-34.603722,-58.4000,29
4,Australia,-35.473500,149.0124,1999


In [ ]:
df_countries2.to_csv("genomics_country_metadata.csv",index=False)